# Lab 2.2.2: Transfer Learning Project

**Module:** 2.2 - Computer Vision  
**Time:** 3 hours  
**Difficulty:** ⭐⭐⭐

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand when and why to use transfer learning
- [ ] Compare feature extraction vs fine-tuning approaches
- [ ] Fine-tune EfficientNet to achieve >90% accuracy on custom data
- [ ] Implement proper learning rate strategies for transfer learning

---

## 📚 Prerequisites

- Completed: Lab 2.2.1 (CNN Architecture Study)
- Knowledge of: PyTorch training loops, data augmentation

---

## 🌍 Real-World Context

**Transfer learning is how most real AI systems are built:**

- 🏥 **Medical imaging**: Start with ImageNet model, fine-tune on X-rays
- 🛍️ **E-commerce**: Classify products using pre-trained features
- 🌾 **Agriculture**: Detect crop diseases with minimal training data
- 🏭 **Manufacturing**: Quality control with few defect examples

**Why?** Training from scratch requires millions of images. Transfer learning needs just hundreds!

---

## 🧒 ELI5: What is Transfer Learning?

> **Imagine you already know how to ride a bicycle...** 🚲
>
> When you learn to ride a motorcycle, do you start from zero? No! You already know:
> - How to balance
> - How to steer
> - How to watch the road
>
> You just need to learn the new parts (throttle, clutch, extra weight).
>
> **Transfer learning is the same for neural networks:**
> - A model trained on ImageNet already knows edges, textures, shapes, objects
> - To classify YOUR images, you just teach it what YOUR specific categories look like
> - This is way faster than teaching it everything from scratch!
>
> **In AI terms:** Pre-trained networks have learned universal visual features. We leverage these features and only train the task-specific layers.

---

## Part 1: Understanding Transfer Learning Strategies

### Two Main Approaches

```
Feature Extraction:                    Fine-Tuning:
┌──────────────────┐                  ┌──────────────────┐
│  Pre-trained     │ ❄️ FROZEN        │  Pre-trained     │ 🔥 TRAINABLE
│  Backbone        │ (no training)    │  Backbone        │ (small LR)
├──────────────────┤                  ├──────────────────┤
│  New Classifier  │ 🔥 TRAINABLE     │  New Classifier  │ 🔥 TRAINABLE
│  Head            │                  │  Head            │ (larger LR)
└──────────────────┘                  └──────────────────┘

When to use:                          When to use:
- Small dataset (<1000 images)        - Medium dataset (1000+)
- Similar to ImageNet                 - Different from ImageNet
- Limited compute                     - Better accuracy needed
```

### The Transfer Learning Spectrum

```
Your data is like ImageNet    ←────────────────────────→    Your data is very different
        │                                                              │
        ▼                                                              ▼
  Feature Extraction                                              Full Fine-tuning
  (freeze everything)                                            (train everything)
```

In [ ]:
# Setup
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, random_split
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple, Dict, List, Optional
from tqdm.auto import tqdm
import time
from pathlib import Path

# DGX Spark optimizations
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

---

## Part 2: The CIFAR-100 Dataset

We'll use CIFAR-100, which has 100 classes - a more challenging task than CIFAR-10. This simulates real-world scenarios where you have many categories and limited training data per class.

In [ ]:
def get_cifar100_loaders(
    batch_size: int = 64,
    num_workers: int = 4,
    subset_size: Optional[int] = None
) -> Tuple[DataLoader, DataLoader]:
    """
    Create CIFAR-100 data loaders with ImageNet-style preprocessing.
    
    Args:
        batch_size: Batch size for training
        num_workers: Number of data loading workers
        subset_size: If specified, use only this many training samples
    
    Returns:
        Tuple of (train_loader, test_loader)
    """
    # ImageNet normalization (required for pre-trained models!)
    imagenet_mean = [0.485, 0.456, 0.406]
    imagenet_std = [0.229, 0.224, 0.225]
    
    # Training transforms with augmentation
    train_transform = transforms.Compose([
        transforms.Resize(256),                          # Upscale for better features
        transforms.RandomResizedCrop(224),               # Standard ImageNet input size
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),     # Color augmentation
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
    ])
    
    # Validation transforms (no augmentation)
    val_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
    ])
    
    # Load datasets
    train_dataset = torchvision.datasets.CIFAR100(
        root='../data', train=True, download=True, transform=train_transform
    )
    test_dataset = torchvision.datasets.CIFAR100(
        root='../data', train=False, download=True, transform=val_transform
    )
    
    # Optionally use subset for faster experiments
    if subset_size:
        indices = torch.randperm(len(train_dataset))[:subset_size]
        train_dataset = Subset(train_dataset, indices)
    
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size,
        shuffle=True, num_workers=num_workers, pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size,
        shuffle=False, num_workers=num_workers, pin_memory=True
    )
    
    return train_loader, test_loader

# Load data (using subset for demonstration)
train_loader, test_loader = get_cifar100_loaders(batch_size=64, subset_size=10000)

print(f"📊 CIFAR-100 Dataset:")
print(f"   Training samples: {len(train_loader.dataset):,}")
print(f"   Test samples:     {len(test_loader.dataset):,}")
print(f"   Classes:          100")
print(f"   Batch size:       64")

In [ ]:
# Visualize some samples
def show_batch(loader, classes_map=None):
    """Display a batch of images with labels."""
    images, labels = next(iter(loader))
    
    # Denormalize for visualization
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    images_vis = images[:16] * std + mean
    images_vis = images_vis.clamp(0, 1)
    
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    for idx, ax in enumerate(axes.flat):
        img = images_vis[idx].permute(1, 2, 0).numpy()
        ax.imshow(img)
        ax.set_title(f'Class: {labels[idx].item()}', fontsize=10)
        ax.axis('off')
    
    plt.suptitle('🖼️ CIFAR-100 Sample Batch (Augmented)', fontsize=14)
    plt.tight_layout()
    plt.show()

show_batch(train_loader)

---

## Part 3: Exploring Pre-trained Models

The `torchvision.models` and `timm` libraries provide many pre-trained models. Let's explore what's available.

In [ ]:
# Install timm if not available
# Note: For DGX Spark, timm is recommended to be installed in your NGC container
# See README.md for Quick Start instructions

try:
    import timm
    print("✅ timm library available")
except ImportError:
    print("⚠️ timm not found. Installing...")
    print("   💡 TIP: For DGX Spark ARM64, pre-install in NGC container for best results:")
    print("      pip install timm")
    !pip install timm -q
    import timm
    print("✅ timm installed successfully!")

# List popular models
print("\n📚 Popular Pre-trained Models for Transfer Learning:")
print("="*60)

popular_models = {
    'ResNet-50': ('resnet50', 25.6),
    'EfficientNet-B0': ('efficientnet_b0', 5.3),
    'EfficientNet-B3': ('efficientnet_b3', 12.2),
    'ConvNeXt-Tiny': ('convnext_tiny', 28.6),
    'ViT-B/16': ('vit_base_patch16_224', 86.6),
}

print(f"{'Model':<20} {'Parameters':>15} {'ImageNet Top-1':>15}")
print("-"*60)

# Top-1 accuracies (approximate)
accuracies = {
    'ResNet-50': 76.1,
    'EfficientNet-B0': 77.1,
    'EfficientNet-B3': 81.6,
    'ConvNeXt-Tiny': 82.1,
    'ViT-B/16': 81.8,
}

for name, (model_name, params) in popular_models.items():
    acc = accuracies.get(name, 'N/A')
    print(f"{name:<20} {params:>12.1f}M {acc:>14.1f}%")

### 🧒 ELI5: Why EfficientNet?

> **Imagine you're building a LEGO tower...**
>
> - **ResNet**: Makes the tower taller (more layers)
> - **Wide ResNet**: Makes each floor bigger (more channels)
> - **EfficientNet**: Carefully balances height, width, AND image resolution
>
> The EfficientNet authors used neural architecture search to find the optimal balance. Result: **Better accuracy with fewer parameters!**

### EfficientNet Scaling

```
EfficientNet-B0 (baseline):  224×224 input,  5.3M params
EfficientNet-B1:             240×240 input,  7.8M params
EfficientNet-B2:             260×260 input,  9.2M params  
EfficientNet-B3:             300×300 input, 12.2M params  ← Sweet spot!
EfficientNet-B4:             380×380 input, 19.3M params
...
EfficientNet-B7:             600×600 input, 66.3M params
```

---

## Part 4: Feature Extraction (Frozen Backbone)

First, let's try the simplest approach: freeze the pre-trained backbone and only train a new classifier head.

In [ ]:
def create_feature_extractor(num_classes: int = 100) -> nn.Module:
    """
    Create EfficientNet-B0 with frozen backbone for feature extraction.
    
    Args:
        num_classes: Number of output classes
    
    Returns:
        Model with frozen backbone and new classifier
    """
    # Load pre-trained EfficientNet-B0
    model = timm.create_model('efficientnet_b0', pretrained=True)
    
    # Freeze ALL backbone parameters
    for param in model.parameters():
        param.requires_grad = False
    
    # Replace classifier head (this stays trainable)
    num_features = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.3),
        nn.Linear(num_features, num_classes)
    )
    
    # Count trainable parameters
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    
    print(f"📊 Feature Extractor Created:")
    print(f"   Total parameters:     {total:,}")
    print(f"   Trainable parameters: {trainable:,}")
    print(f"   Frozen parameters:    {total - trainable:,}")
    print(f"   Trainable ratio:      {100*trainable/total:.2f}%")
    
    return model

feature_extractor = create_feature_extractor(num_classes=100)

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    running_loss, correct, total = 0, 0, 0
    
    for inputs, targets in tqdm(loader, desc='Training', leave=False):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    return running_loss / total, 100 * correct / total


def evaluate(model, loader, criterion, device):
    """Evaluate model accuracy."""
    model.eval()
    running_loss, correct, total = 0, 0, 0
    
    with torch.no_grad():
        for inputs, targets in tqdm(loader, desc='Evaluating', leave=False):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    return running_loss / total, 100 * correct / total


def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    epochs: int = 10,
    lr: float = 0.001,
    device: torch.device = device
) -> Dict[str, List[float]]:
    """
    Train a model with standard settings.
    
    Returns:
        Dictionary of training history
    """
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    # Only optimize trainable parameters
    optimizer = optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=lr, weight_decay=0.01
    )
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        scheduler.step()
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"   Train: Loss={train_loss:.4f}, Acc={train_acc:.1f}%")
        print(f"   Val:   Loss={val_loss:.4f}, Acc={val_acc:.1f}%")
    
    return history

In [ ]:
# Train feature extractor (fast, but limited accuracy)
print("🧊 Training Feature Extractor (Frozen Backbone)...")
print("="*50)

feature_extractor = create_feature_extractor(num_classes=100)
start_time = time.time()

fe_history = train_model(
    feature_extractor, 
    train_loader, 
    test_loader,
    epochs=5,  # Quick training since backbone is frozen
    lr=0.01    # Can use higher LR since only training head
)

fe_time = time.time() - start_time
print(f"\n✅ Feature extraction complete in {fe_time:.1f}s")
print(f"   Final accuracy: {fe_history['val_acc'][-1]:.1f}%")

---

## Part 5: Full Fine-Tuning

Now let's train the entire network, including the backbone. This requires careful learning rate management.

In [ ]:
def create_finetune_model(num_classes: int = 100) -> nn.Module:
    """
    Create EfficientNet-B0 for full fine-tuning.
    
    All parameters are trainable, but we'll use different learning rates.
    """
    # Load pre-trained model
    model = timm.create_model('efficientnet_b0', pretrained=True)
    
    # Replace classifier
    num_features = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.3),
        nn.Linear(num_features, num_classes)
    )
    
    # ALL parameters are trainable
    total = sum(p.numel() for p in model.parameters())
    print(f"📊 Fine-tuning Model Created:")
    print(f"   Total/Trainable parameters: {total:,}")
    
    return model

finetune_model = create_finetune_model(num_classes=100)

### 🧒 ELI5: Differential Learning Rates

> **Imagine you're renovating a house...** 🏠
>
> - The **foundation** (early layers): Already solid! Only minor adjustments needed.
> - The **walls** (middle layers): Might need some changes to fit your style.
> - The **furniture** (classifier head): Brand new! Built specifically for you.
>
> Similarly, we use **smaller learning rates for early layers** (small adjustments to solid foundation) and **larger learning rates for the classifier** (learning from scratch).

```
Learning Rate Strategy:

┌─────────────────┐
│  Classifier     │  LR = 0.001   (large, learning new task)
├─────────────────┤
│  Late Layers    │  LR = 0.0001  (medium, adapt features)
├─────────────────┤
│  Early Layers   │  LR = 0.00001 (tiny, preserve general features)
└─────────────────┘
```

In [ ]:
def train_with_discriminative_lr(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    epochs: int = 10,
    base_lr: float = 0.0001,
    device: torch.device = device
) -> Dict[str, List[float]]:
    """
    Train with discriminative (layer-wise) learning rates.
    
    Backbone gets smaller LR, classifier gets larger LR.
    """
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    # Create parameter groups with different LRs
    # EfficientNet structure: conv_stem -> blocks -> conv_head -> classifier
    
    # Separate backbone and classifier parameters
    backbone_params = []
    classifier_params = []
    
    for name, param in model.named_parameters():
        if 'classifier' in name:
            classifier_params.append(param)
        else:
            backbone_params.append(param)
    
    optimizer = optim.AdamW([
        {'params': backbone_params, 'lr': base_lr},
        {'params': classifier_params, 'lr': base_lr * 10}  # 10× for classifier
    ], weight_decay=0.01)
    
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        scheduler.step()
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        current_lrs = [group['lr'] for group in optimizer.param_groups]
        print(f"   Train: Loss={train_loss:.4f}, Acc={train_acc:.1f}%")
        print(f"   Val:   Loss={val_loss:.4f}, Acc={val_acc:.1f}%")
        print(f"   LRs:   backbone={current_lrs[0]:.6f}, classifier={current_lrs[1]:.6f}")
    
    return history

In [ ]:
# Train with full fine-tuning
print("🔥 Training Full Fine-Tuning (Discriminative LR)...")
print("="*50)

finetune_model = create_finetune_model(num_classes=100)
start_time = time.time()

ft_history = train_with_discriminative_lr(
    finetune_model,
    train_loader,
    test_loader,
    epochs=10,
    base_lr=0.0001
)

ft_time = time.time() - start_time
print(f"\n✅ Fine-tuning complete in {ft_time:.1f}s")
print(f"   Final accuracy: {ft_history['val_acc'][-1]:.1f}%")

---

## Part 6: Gradual Unfreezing Strategy

A more sophisticated approach: start with frozen backbone, then progressively unfreeze layers.

In [ ]:
def train_with_gradual_unfreeze(
    num_classes: int = 100,
    train_loader: DataLoader = train_loader,
    val_loader: DataLoader = test_loader,
    device: torch.device = device
) -> Dict[str, List[float]]:
    """
    Train using gradual unfreezing:
    1. Train only classifier (2 epochs)
    2. Unfreeze last 2 blocks + classifier (3 epochs)
    3. Unfreeze all (5 epochs)
    """
    # Create model with frozen backbone
    model = timm.create_model('efficientnet_b0', pretrained=True)
    
    # Replace classifier
    num_features = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.3),
        nn.Linear(num_features, num_classes)
    )
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'phase': []}
    
    # ========== Phase 1: Train classifier only ==========
    print("\n" + "="*50)
    print("🧊 Phase 1: Training classifier only (backbone frozen)")
    print("="*50)
    
    # Freeze everything
    for param in model.parameters():
        param.requires_grad = False
    
    # Unfreeze classifier
    for param in model.classifier.parameters():
        param.requires_grad = True
    
    optimizer = optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=0.01
    )
    
    for epoch in range(2):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['phase'].append('classifier')
        
        print(f"Epoch {epoch+1}: Train Acc={train_acc:.1f}%, Val Acc={val_acc:.1f}%")
    
    # ========== Phase 2: Unfreeze last 2 blocks ==========
    print("\n" + "="*50)
    print("🌡️ Phase 2: Unfreezing last 2 blocks")
    print("="*50)
    
    # EfficientNet-B0 has blocks named 'blocks.X'
    # Unfreeze blocks 5 and 6 (last two)
    for name, param in model.named_parameters():
        if 'blocks.5' in name or 'blocks.6' in name or 'conv_head' in name:
            param.requires_grad = True
    
    # New optimizer with lower LR for unfrozen layers
    optimizer = optim.AdamW([
        {'params': [p for n, p in model.named_parameters() 
                    if ('blocks.5' in n or 'blocks.6' in n or 'conv_head' in n) and p.requires_grad], 
         'lr': 0.0001},
        {'params': model.classifier.parameters(), 'lr': 0.001}
    ])
    
    for epoch in range(3):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['phase'].append('partial')
        
        print(f"Epoch {epoch+3}: Train Acc={train_acc:.1f}%, Val Acc={val_acc:.1f}%")
    
    # ========== Phase 3: Unfreeze all ==========
    print("\n" + "="*50)
    print("🔥 Phase 3: Full fine-tuning")
    print("="*50)
    
    # Unfreeze everything
    for param in model.parameters():
        param.requires_grad = True
    
    # Very small LR for early layers
    optimizer = optim.AdamW(model.parameters(), lr=0.00001)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)
    
    for epoch in range(5):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        scheduler.step()
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['phase'].append('full')
        
        print(f"Epoch {epoch+6}: Train Acc={train_acc:.1f}%, Val Acc={val_acc:.1f}%")
    
    return history, model

In [ ]:
# Train with gradual unfreezing
print("🎓 Training with Gradual Unfreezing Strategy...")
start_time = time.time()

gu_history, best_model = train_with_gradual_unfreeze()

gu_time = time.time() - start_time
print(f"\n✅ Gradual unfreezing complete in {gu_time:.1f}s")
print(f"   Final accuracy: {gu_history['val_acc'][-1]:.1f}%")

---

## Part 7: Compare All Approaches

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Validation accuracy
axes[0].plot(fe_history['val_acc'], label='Feature Extraction', linewidth=2, marker='o')
axes[0].plot(ft_history['val_acc'], label='Full Fine-tuning', linewidth=2, marker='s')
axes[0].plot(gu_history['val_acc'], label='Gradual Unfreezing', linewidth=2, marker='^')

# Mark phase transitions for gradual unfreezing
phases = gu_history['phase']
for i, phase in enumerate(phases):
    if i > 0 and phases[i] != phases[i-1]:
        axes[0].axvline(x=i, color='gray', linestyle='--', alpha=0.5)

axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Validation Accuracy (%)')
axes[0].set_title('📈 Validation Accuracy Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Training loss
axes[1].plot(fe_history['train_loss'], label='Feature Extraction', linewidth=2)
axes[1].plot(ft_history['train_loss'], label='Full Fine-tuning', linewidth=2)
axes[1].plot(gu_history['train_loss'], label='Gradual Unfreezing', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Training Loss')
axes[1].set_title('📉 Training Loss Comparison')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "="*60)
print("📊 TRANSFER LEARNING COMPARISON")
print("="*60)
print(f"{'Approach':<25} {'Final Acc':>12} {'Best Acc':>12} {'Time':>10}")
print("-"*60)
print(f"{'Feature Extraction':<25} {fe_history['val_acc'][-1]:>11.1f}% {max(fe_history['val_acc']):>11.1f}% {fe_time:>9.1f}s")
print(f"{'Full Fine-tuning':<25} {ft_history['val_acc'][-1]:>11.1f}% {max(ft_history['val_acc']):>11.1f}% {ft_time:>9.1f}s")
print(f"{'Gradual Unfreezing':<25} {gu_history['val_acc'][-1]:>11.1f}% {max(gu_history['val_acc']):>11.1f}% {gu_time:>9.1f}s")

### ✋ Try It Yourself

Experiment with different configurations:

1. **Try a larger model**: Replace `efficientnet_b0` with `efficientnet_b3`
2. **Adjust augmentation**: Add/remove transforms and see the effect
3. **Use the full dataset**: Remove the `subset_size` parameter

<details>
<summary>💡 Hint for achieving >90% accuracy</summary>

To hit 90%+ on CIFAR-100:
- Use the full 50,000 training images
- Try EfficientNet-B3 or ConvNeXt
- Train for 20+ epochs with gradual unfreezing
- Add stronger augmentation (Mixup, CutMix)
- Use label smoothing: `nn.CrossEntropyLoss(label_smoothing=0.1)`

</details>

---

## ⚠️ Common Mistakes

### Mistake 1: Forgetting to normalize inputs

```python
# ❌ Wrong: No normalization
transform = transforms.ToTensor()

# ✅ Right: Use ImageNet normalization for pre-trained models
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])
```
**Why:** Pre-trained models expect inputs normalized to ImageNet statistics.

### Mistake 2: Same learning rate for all layers

```python
# ❌ Wrong: Same high LR for everything
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ Right: Lower LR for pretrained layers
optimizer = optim.Adam([
    {'params': model.backbone.parameters(), 'lr': 1e-5},
    {'params': model.classifier.parameters(), 'lr': 1e-3}
])
```
**Why:** High LR destroys pretrained weights. Lower LR preserves good features.

### Mistake 3: Not replacing the classifier head

```python
# ❌ Wrong: Training on 1000-class head
model = models.resnet50(pretrained=True)  # 1000 classes!

# ✅ Right: Replace with correct number of classes
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Your classes
```
**Why:** ImageNet has 1000 classes. You need to match your dataset.

---

## 🎉 Checkpoint

You've learned:
- ✅ When to use feature extraction vs fine-tuning
- ✅ How to load and modify pre-trained models
- ✅ Discriminative learning rates for transfer learning
- ✅ Gradual unfreezing strategy for best results
- ✅ Proper normalization for pre-trained models

---

## 🚀 Challenge (Optional)

**Beat 90% accuracy on CIFAR-100!**

Techniques to try:
1. **Mixup/CutMix augmentation**
2. **Label smoothing**
3. **Larger models** (EfficientNet-B4 or ConvNeXt-Small)
4. **Longer training** with warmup
5. **Test-time augmentation** (TTA)

Record your best accuracy and the techniques used!

In [ ]:
# YOUR CHALLENGE CODE HERE
# Goal: >90% accuracy on CIFAR-100



---

## 📖 Further Reading

- [timm Library Documentation](https://huggingface.co/docs/timm/index)
- [EfficientNet Paper](https://arxiv.org/abs/1905.11946)
- [How to Train Your ViT](https://arxiv.org/abs/2106.10270)
- [Transfer Learning in Computer Vision](https://cs231n.github.io/transfer-learning/)

---

## 🧹 Cleanup

In [ ]:
# Clear GPU memory
import gc

del feature_extractor, finetune_model, best_model
torch.cuda.empty_cache()
gc.collect()

print("✅ Cleanup complete!")
if torch.cuda.is_available():
    print(f"💾 GPU Memory Free: {torch.cuda.mem_get_info()[0] / 1e9:.1f} GB")